# 术语名词解释

## DTS(Decoding Time Stamp) 和 PTS(Presentation Time Stamp)

H264里有两种时间戳：DTS（Decoding Time Stamp）和PTS（Presentation Time Stamp）。

DTS 指的是解码时间 
PTS 指的是显示时间 

FFmpeg 中用 AVPacket 结构体来描述解码前或编码后的压缩包，
用AVFrame结构体来描述解码后或编码前的信号帧。

对于视频来说，AVFrame就是视频的一帧图像。
- 这帧图像什么时候显示给用户，就取决于它的PTS。
- DTS是AVPacket里的一个成员，表示这个压缩包应该什么时候被解码。

>需要注意的是, 如果视频里各帧的编码是按输入顺序（也就是显示顺序）依次进行的，
>那么解码和显示时间应该是一致的。

>可事实上，在大多数编解码标准（如H.264或HEVC）中，编码顺序和输入顺序并不一致。 
>于是才会需要PTS和DTS这两种不同的时间戳。

# MP4文件格式概览

MP4文件由多个box组成，每个box存储不同的信息，且box之间是树状结构，如下图所示。

![图 2](../asset/fa715aef84b6d082e1d20e6fb25febbc497fbd4150989dcd6036187d18815fe2.png)  


box类型有很多，下面是3个比较重要的顶层box：

- ftyp：File Type Box，描述文件遵从的MP4规范与版本；
- moov：Movie Box，媒体的metadata信息，有且仅有一个。
- mdat：Media Data Box，存放实际的媒体数据，一般有多个；

![图 1](../asset/ae490519726a0c4d0d415d2fa513ffe6c5f82b439b68fca7e9808445f03fa51d.png)  

下面是一个典型的MP4文件结构:

![图 3](../asset/9c0e13f30a4e0ccacab680a85eba8d4c1ca9c903258b90f5f32592a35bcb3ce6.png)  


下面是一些常见的Box的简介:

![图 4](../asset/56baa44dd65fd084b02f3e0a5f4a4b74ce859d947aa70f8c73f6837277e2e2d7.png)  


# MP4 Box简介
1个box由两部分组成：box header、box body。

- box header：box的元数据，比如box type、box size。
- box body：box的数据部分，实际存储的内容跟box类型有关，比如mdat中body部分存储的媒体数据。

box header中，只有type、size是必选字段。
当size==0时，存在largesize字段。
在部分box中，还存在version、flags字段，这样的box叫做Full Box。
当box body中嵌套其他box时，这样的box叫做container box。

![图 5](../asset/da1ba53ba86b0f6a5be0c9d6b55baeb73d3200ec3eaf647ec7cd6e7729f3501a.png)  


## Box Header
字段定义如下：

- type：box类型，包括 “预定义类型”、“自定义扩展类型”，占4个字节；
    - 预定义类型：比如ftyp、moov、mdat等预定义好的类型；
    - 自定义扩展类型：如果type==uuid，则表示是自定义扩展类型。size（或largesize）随后的16字节，为自定义类型的值（extended_type）
- size：包含box header在内的整个box的大小，单位是字节。当size为0或1时，需要特殊处理：
    - size等于0：box的大小由后续的largesize确定（一般只有装载媒体数据的mdat box会用到largesize）；
    - size等于1：当前box为文件的最后一个box，通常包含在mdat box中；
- largesize：box的大小，占8个字节；
- extended_type：自定义扩展类型，占16个字节；

In [146]:
from construct import *

f = open("./data/data1.mp4", "rb")
data = f.read()
f.close()


class Box:
    def __init__(self, name=''):
        if not hasattr(self, 'body_fmt'):
            self.body_fmt = IfThenElse(
                this.size == 1,
                GreedyBytes,
                Bytes(
                    this.size - 8 if this.large_size == None else this.large_size - 16
                ),
            )

        self.struct = None
        self.size = None
        self.body = None
        self.box_name = name

        self.fmt = Struct(
            "size" / Int32ub,
            "type" / PaddedString(4, "ascii"),
            "large_size" / Optional(If(this.size == 0, Int64ub)),
            "extended_type" / Optional(If(this.type == "uuid", Bytes(16))),
            "body" / self.body_fmt,
        )

    def __str__(self):
        ret = f"box_name: {self.box_name}\nNormal Box " + self.struct.__str__()
        return ret

    def init(self, data):
        self.struct = self.fmt.parse(data)
        self.size = (
            self.struct.size
            if self.struct.large_size == None
            else self.struct.large_size
        )
        if self.box_name == '':
            self.box_name = self.struct.type
        self.body = self.struct.body

    @staticmethod
    def getBoxList(data):
        cur_offset = 0
        boxes = []
        data_len = len(data)

        while cur_offset < data_len:
            tmp = Box(data[cur_offset:])

            # print(f"Current pos is {cur_offset}/{data_len}")

            box = None
            if tmp.box_name == "avc1":
                box = VisualSampleEntry(data[cur_offset:])
            else:
                box = tmp

            boxes.append(box)
            cur_offset += box.size

        return boxes


box = Box()
box.init(data)
print(box)


box_name: ftyp
Normal Box Container: 
    size = 32
    type = u'ftyp' (total 4)
    large_size = None
    extended_type = None
    body = b'isom\x00\x00\x02\x00isomiso2'... (truncated, total 24)


## FullBox
在Box的基础上，扩展出了FullBox类型。相比Box，FullBox 多了 version、flags 字段。

- version：当前box的版本，为扩展做准备，占1个字节；
- flags：标志位，占24位，含义由具体的box自己定义；

FullBox主要在moov中的box用到，比如 moov.mvhd

In [147]:
class FullBox(Box):
    def __init__(self, name=''):
        if not hasattr(self, 'body_fmt'):
            self.body_fmt = IfThenElse(
                this.size == 1,
                GreedyBytes,
                Bytes(
                    this.size - 12 if this.large_size == None else this.large_size - 20
                ),
            )

        super().__init__(name)

        self.version = None
        self.flags = None

        self.fmt = Struct(
            "size" / Int32ub,
            "type" / PaddedString(4, "ascii"),
            "large_size" / Optional(If(this.size == 0, Int64ub)),
            "extended_type" / Optional(If(this.type == "uuid", Bytes(16))),
            "version" / Int8ub,
            "flags" / Bytes(3),
            "body" / self.body_fmt,
        )

    def init(self, data):
        super().init(data)
        self.version = self.struct.version
        self.flags = self.struct.flags

    def __str__(self):
        ret = f"box_name: {self.box_name}\nFull Box " + self.struct.__str__()
        return ret


# ftyp(File Type Box)
ftyp用来指出当前文件遵循的规范

## 什么是isom
isom（ISO Base Media file）是在 MPEG-4 Part 12 中定义的一种基础文件格式，MP4、3gp、QT 等常见的封装格式，都是基于这种基础文件格式衍生的。

MP4 文件可能遵循的规范有mp41、mp42，而mp41、mp42又是基于isom衍生出来的。

>3gp(3GPP)：一种容器格式，主要用于3G手机上；  
>QT：QuickTime的缩写，.qt 文件代表苹果QuickTime媒体文件；

## ftyp 定义
其头部信息和 Box 一致, 主要是 body 部分:
- major_brand(4 字节字符串): 比如常见的 isom、mp41、mp42、avc1、qt等。它表示“最好”基于哪种格式来解析当前的文件。
- minor_version(4 字节无符号整数): 提供 major_brand 的说明信息，比如版本号，不得用来判断媒体文件是否符合某个标准/规范
- compatible_brands(4 字节字符串列表): 文件兼容的brand列表。比如 mp41 的兼容 brand 为 isom。通过兼容列表里的 brand 规范，可以将文件 部分（或全部）解码出来；

>在实际使用中，不能把 isom 做为 major_brand，而是需要使用具体的brand（比如mp41），  
>因此，对于 isom，没有定义具体的文件扩展名、mime type。

下面是常见的几种brand，以及对应的文件扩展名、mime type，更多brand可以参考 [这里](http://fileformats.archiveteam.org/wiki/Boxes/atoms_format#Brands) 

![图 6](../asset/b3cfd18009831c5b910c8fa048d1ed88785f6f6f87e5e3f9a5f8c34872422979.png)  




In [148]:
class FileTypeBox(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "major_brand" / PaddedString(4, "ascii"),
            "minor_version" / Int32ub,
            "compatible_brands" / GreedyRange(PaddedString(4, "ascii")),
        )

        super().__init__('ftyp')


ftyp = FileTypeBox()
ftyp.init(data)
print(ftyp)


box_name: ftyp
Normal Box Container: 
    size = 32
    type = u'ftyp' (total 4)
    large_size = None
    extended_type = None
    body = Container: 
        major_brand = u'isom' (total 4)
        minor_version = 512
        compatible_brands = ListContainer: 
            isom
            iso2
            avc1
            mp41


# moov(Movie Box)

Movie Box，存储 mp4 的 metadata，一般位于mp4文件的开头。

moov中，最重要的两个box是 mvhd 和 trak：

- mvhd：Movie Header Box，mp4文件的整体信息，比如创建时间、文件时长等；
- trak：Track Box，一个mp4可以包含一个或多个轨道（比如视频轨道、音频轨道），轨道相关的信息就在trak里。trak是container box，至少包含两个box，tkhd、mdia；

>mvhd针对整个影片，tkhd针对单个track，mdhd针对媒体，vmhd针对视频，smhd针对音频，  
>可以认为是从 宽泛 > 具体，前者一般是从后者推导出来的。

## mvhd(Movie Header Box)

MP4文件的整体信息，跟具体的视频流、音频流无关，比如创建时间、文件时长等。

定义如下:
```java
aligned(8) class MovieHeaderBox extends FullBox(‘mvhd’, version, 0) { 
    if (version==1) {
      unsigned int(64)  creation_time;
      unsigned int(64)  modification_time;
      unsigned int(32)  timescale;
      unsigned int(64)  duration;
    } else { // version==0
      unsigned int(32)  creation_time;
      unsigned int(32)  modification_time;
      unsigned int(32)  timescale;
      unsigned int(32)  duration;
    }
    template int(32) rate = 0x00010000; // typically 1.0
    template int(16) volume = 0x0100; // typically, full volume 
    const bit(16) reserved = 0;
    const unsigned int(32)[2] reserved = 0;
    template int(32)[9] matrix =
        { 0x00010000,0,0,0,0x00010000,0,0,0,0x40000000 };
        // Unity matrix
    bit(32)[6]  pre_defined = 0;
    unsigned int(32)  next_track_ID;
}
```
含义如下:

- creation_time：文件创建时间；
- modification_time：文件修改时间；
- timescale：一秒包含的时间单位（整数）。举个例子，如果timescale等于1000，那么，一秒包含1000个时间单位（后面track等的时间，都要用这个来换算，比如track的duration为10,000，那么，track的实际时长为10,000/1000=10s）；
- duration：影片时长（整数），根据文件中的track的信息推导出来，等于时间最长的track的duration；
- rate：推荐的播放速率，32位整数，高16位、低16位分别代表整数部分、小数部分（[16.16]），举例 0x0001 0000 代表1.0，正常播放速度；
- volume：播放音量，16位整数，高8位、低8位分别代表整数部分、小数部分（[8.8]），举例 0x01 00 表示 1.0，即最大音量；
- matrix：视频的转换矩阵，一般可以忽略不计；
- next_track_ID：32位整数，非0，一般可以忽略不计。当要添加一个新的track到这个影片时，可以使用的track id，必须比当前已经使用的track id要大。也就是说，添加新的track时，需要遍历所有track，确认可用的track id；

In [149]:
class MovieHeaderBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "creation_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "modification_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "timescale" / Int32ub,
            "duration" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "rate" / Bytes(4),
            "volume" / Bytes(2),
            "reserved1" / Bytes(2),
            "reserved2" / Bytes(8),
            "matrix" / Bytes(4)[9],
            "pre_defined" / Bytes(4)[6],
            "next_track_ID" / Int32ub,
        )

        super().__init__('mvhd')

    def init(self, data):
        super().init(data)
        self.body.rate = self.convertRate(self.body.rate)
        self.body.volume = self.convertVolume(self.body.volume)

    def convertRate(self, rate_byte):
        high = int.from_bytes(rate_byte[:2], byteorder="big")
        low = int.from_bytes(rate_byte[2:], byteorder="big")
        return str(high) + "." + str(low)

    def convertVolume(self, volume_byte):
        high = int.from_bytes(volume_byte[:1], byteorder="big")
        low = int.from_bytes(volume_byte[1:], byteorder="big")
        return str(high) + "." + str(low)


mvhd_offset = data.find(b"mvhd") - 4
mvhd = MovieHeaderBox()
mvhd.init(data[mvhd_offset:])
# print(mvhd.body)
print(mvhd)


box_name: mvhd
Full Box Container: 
    size = 108
    type = u'mvhd' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        creation_time = 0
        modification_time = 0
        timescale = 1000
        duration = 134768
        rate = u'1.0' (total 3)
        volume = u'1.0' (total 3)
        reserved1 = b'\x00\x00' (total 2)
        reserved2 = b'\x00\x00\x00\x00\x00\x00\x00\x00' (total 8)
        matrix = ListContainer: 
            b'\x00\x01\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x01\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'@\x00\x00\x00'
        pre_defined = ListContainer: 
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'

## trak(Track Box)

Track Box 是一个 Container Box, 里面包含至少两个 Box, tkhd(Track Header Box), media
一个mp4可以包含一个或多个轨道（比如视频轨道、音频轨道），轨道相关的信息就在 trak 里。

定义如下: 
```java
aligned(8) class TrackBox extends Box(‘trak’) { 
} 
```

### thkd(Track Header Box)

定义如下:

```java
aligned(8) class TrackHeaderBox 
  extends FullBox(‘tkhd’, version, flags){ 
	if (version==1) {
	      unsigned int(64)  creation_time;
	      unsigned int(64)  modification_time;
	      unsigned int(32)  track_ID;
	      const unsigned int(32)  reserved = 0;
	      unsigned int(64)  duration;
	   } else { // version==0
	      unsigned int(32)  creation_time;
	      unsigned int(32)  modification_time;
	      unsigned int(32)  track_ID;
	      const unsigned int(32)  reserved = 0;
	      unsigned int(32)  duration;
	}
	const unsigned int(32)[2] reserved = 0;
	template int(16) layer = 0;
	template int(16) alternate_group = 0;
	template int(16) volume = {if track_is_audio 0x0100 else 0}; 
    const unsigned int(16) reserved = 0;
	template int(32)[9] matrix= { 0x00010000,0,0,0,0x00010000,0,0,0,0x40000000 }; // unity matrix
	unsigned int(32) width;
	unsigned int(32) height;
}
```

单个 track 的 metadata，包含如下字段：

- version：tkhd box的版本；
- flags：按位或操作获得，默认值是7（0x000001 | 0x000002 | 0x000004），表示这个track是启用的、用于播放的 且 用于预览的。
    - Track_enabled：值为0x000001，表示这个track是启用的，当值为0x000000，表示这个track没有启用；
    - Track_in_movie：值为0x000002，表示当前track在播放时会用到；
    - Track_in_preview：值为0x000004，表示当前track用于预览模式；
- creation_time：当前track的创建时间；
- modification_time：当前track的最近修改时间；
- track_ID：当前track的唯一标识，不能为0，不能重复；
- duration：当前track的完整时长（需要除以timescale得到具体秒数）；
- layer：视频轨道的叠加顺序，数字越小越靠近观看者，比如1比2靠上，0比1靠上；
- alternate_group：当前track的分组ID，alternate_group值相同的track在同一个分组里面。同个分组里的track，同一时间只能有一个track处于播放状态。当alternate_group为0时，表示当前track没有跟其他track处于同个分组。一个分组里面，也可以只有一个track；
- volume：audio track的音量，介于0.0~1.0之间；
- matrix：视频的变换矩阵；
- width、height：视频的宽高；

In [150]:
class TrackBox(Box):
    def __init__(self):
        super().__init__('trak')


class TrackHeaderBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "creation_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "modification_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "track_ID" / Int32ub,
            "reserved1" / Int32ub,
            "duration" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "reserved2" / Bytes(8),
            "layer" / Int16sb,
            "alternate_group" / Int16sb,
            "volume" / Bytes(2),
            "reserved3" / Bytes(2),
            "matrix" / Bytes(4)[9],
            "width" / Aligned(4, Int16ub),
            "height" / Aligned(4, Int16ub),
        )

        super().__init__('thkd')

    def init(self, data):
        super().init(data)

        self.body.volume = self.convertVolume(self.body.volume)

    def convertVolume(self, volume_byte):
        high = int.from_bytes(volume_byte[:1], byteorder="big")
        low = int.from_bytes(volume_byte[1:], byteorder="big")
        return str(high) + "." + str(low)


trak_offset = data.find(b"trak") - 4
trak = TrackBox()
trak.init(data[trak_offset:])
print(trak)
print()
thkd = TrackHeaderBox()
thkd.init(trak.body)
# print(thkd.body)
print(thkd)


box_name: trak
Normal Box Container: 
    size = 54678
    type = u'trak' (total 4)
    large_size = None
    extended_type = None
    body = b'\x00\x00\x00\\tkhd\x00\x00\x00\x03\x00\x00\x00\x00'... (truncated, total 54670)

box_name: thkd
Full Box Container: 
    size = 92
    type = u'tkhd' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x03' (total 3)
    body = Container: 
        creation_time = 0
        modification_time = 0
        track_ID = 1
        reserved1 = 0
        duration = 134668
        reserved2 = b'\x00\x00\x00\x00\x00\x00\x00\x00' (total 8)
        layer = 0
        alternate_group = 0
        volume = u'0.0' (total 3)
        reserved3 = b'\x00\x00' (total 2)
        matrix = ListContainer: 
            b'\x00\x01\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x01\x00\x00'
            b'\x00\x00\x00\x00'
            b'\x00\x00\x00\x00'
  

### mdia()

### mdhd(Media Header Box)

主要是用来存放 track 中视频流创建时间，长度, 时间基准等信息。

![图 3](../asset/cd219425a4e6132393c5dc3314b05deaee986280e25264974bb0ac5b0f748cbb.png)  


```java
aligned(8) class MediaHeaderBox extends FullBox(‘mdhd’, version, 0) { 
 if (version==1) { 
  unsigned int(64)  creation_time; 
  unsigned int(64)  modification_time; 
  unsigned int(32)  timescale; 
  unsigned int(64)  duration; 
 } else { // version==0 
  unsigned int(32)  creation_time; 
  unsigned int(32)  modification_time; 
  unsigned int(32)  timescale; 
  unsigned int(32)  duration; 
 } 
 bit(1) pad = 0; 
 unsigned int(5)[3] language; // ISO-639-2/T language code 
 unsigned int(16)  pre_defined = 0; 
}
```

In [151]:
class MediaHeaderBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "creation_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "modification_time" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "timescale" / Int32ub,
            "duration" / IfThenElse(this._root.version == 1, Int64ub, Int32ub),
            "language_bit" / BitStruct(
                "pad" / Bit,
                "language" / BitsInteger(5)[3],
            ),
            "pre_defined" / Int16ub,
        )

        super().__init__("mdhd")


mdhd_offset = data.find(b"mdhd") - 4
mdhd = MediaHeaderBox()
mdhd.init(data[mdhd_offset:])
print(mdhd)


box_name: mdhd
Full Box Container: 
    size = 32
    type = u'mdhd' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        creation_time = 0
        modification_time = 0
        timescale = 16000
        duration = 2154688
        language_bit = Container: 
            pad = 0
            language = ListContainer: 
                21
                14
                4
        pre_defined = 0


#### hdlr(Handler Reference Box)

声明当前track的类型，以及对应的处理器（handler）。

定义如下:
```java
aligned(8) class HandlerBox extends FullBox(‘hdlr’, version = 0, 0) { 
	unsigned int(32) pre_defined = 0;
	unsigned int(32) handler_type;
	const unsigned int(32)[3] reserved = 0;
   	string   name;
}
```

字段含义如下:
- handler_type: 有如下类型
    - vide（0x76 69 64 65），video track；
    - soun（0x73 6f 75 6e），audio track；
    - hint（0x68 69 6e 74），hint track；
- name: 为utf8字符串，对handler进行描述，比如 L-SMASH Video Handler（参考 [这里](http://avisynth.nl/index.php/LSMASHSource)）。

In [152]:
class HandlerBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "pre_defined" / Int32ub,
            "handler_type" / Bytes(4),
            "reserved" / Bytes(12),
            "name" / CString('utf8')
        )

        super().__init__('hdlr')


hdlr_offset = data.find(b"hdlr") - 4
hdlr = HandlerBox()
hdlr.init(data[hdlr_offset:])
print(hdlr)


box_name: hdlr
Full Box Container: 
    size = 45
    type = u'hdlr' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        pre_defined = 0
        handler_type = b'vide' (total 4)
        reserved = b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' (total 12)
        name = u'VideoHandler' (total 12)


##### stbl(Sample Table Box)

MP4文件的媒体数据部分在mdat box里，而stbl则包含了这些媒体数据的索引以及时间信息，了解stbl对解码、渲染MP4文件很关键。

在SampleDescriptionBox 中，handler_type 参数 为 track 的类型（soun、vide、hint）
entry_count 变量代表当前box中 smaple description 的条目数。

>[stsc](#stscsample-to-chunk-box) 中，sample_description_index 就是指向这些smaple description的索引。

针对不同的[handler_type](#hdlrhandler-reference-box)，
SampleDescriptionBox 后续应用不同的 SampleEntry 类型，
比如video track为VisualSampleEntry。

VisualSampleEntry包含如下字段：

- data_reference_index：当MP4文件的数据部分，可以被分割成多个片段，每一段对应一个索引，并分别通过URL地址来获取，此时，data_reference_index 指向对应的片段（比较少用到）；
- width、height：视频的宽高，单位是像素；
- horizresolution、vertresolution：水平、垂直方向的分辨率（像素/英寸），16.16定点数，默认是0x00480000（72dpi）；
- frame_count：一个sample中包含多少个frame，对video track来说，默认是1；
- compressorname：仅供参考的名字，通常用于展示，占32个字节，比如 AVC Coding。第一个字节，表示这个名字实际要占用N个字节的长度。第2到第N+1个字节，存储这个名字。第N+2到32个字节为填充字节。compressorname 可以设置为0；
- depth：位图的深度信息，比如 0x0018（24），表示不带alpha通道的图片；

定义如下:
```java
aligned(8) class SampleTableBox extends Box(‘stbl’) { 
}
```

在MP4文件中，媒体数据被分成多个chunk，每个chunk可包含多个sample，而sample则由帧组成（通常1个sample对应1个帧），关系如下：

![图 7](../asset/4d5a52a65e7d6643b54f0509b8ba230aa1c56b6fdb9b0c4cf1fe157be4ab4657.jpg)  


stbl中比较关键的box包含stsd、stco、stsc、stsz、stts、stss、ctts。下面先来个概要的介绍，然后再逐个讲解细节。

###### stco / stsc / stsz / stts / stss / ctts / stsd 概述
下面是这几个box概要的介绍：

- stsd：给出视频、音频的编码、宽高、音量等信息，以及每个sample中包含多少个frame；
- stco：thunk在文件中的偏移；
- stsc：每个thunk中包含几个sample；
- stsz：每个sample的size（单位是字节）；
- stts：每个sample的时长；
- stss：哪些sample是关键帧；
- ctts：帧解码到渲染的时间差值，通常用在B帧的场景；


###### stsd（Sample Description Box）

stsd给出sample的描述信息，这里面包含了在解码阶段需要用到的任意初始化信息，比如 编码 等。对于视频、音频来说，所需要的初始化信息不同，这里以视频为例。

定义如下:
```java
aligned(8) abstract class SampleEntry (unsigned int(32) format) 
 extends Box(format){ 
 const unsigned int(8)[6] reserved = 0; 
 unsigned int(16) data_reference_index; 
} 
 
class HintSampleEntry() extends SampleEntry (protocol) { 
 unsigned int(8) data []; 
} 
 // Visual Sequences 
class VisualSampleEntry(codingname) extends SampleEntry (codingname){ 
 unsigned int(16) pre_defined = 0; 
 const unsigned int(16) reserved = 0; 
 unsigned int(32)[3]  pre_defined = 0; 
 unsigned int(16)  width; 
 unsigned int(16)  height; 
 template unsigned int(32)  horizresolution = 0x00480000; // 72 dpi 
 template unsigned int(32)  vertresolution  = 0x00480000; // 72 dpi 
 const unsigned int(32)  reserved = 0; 
 template unsigned int(16)  frame_count = 1; 
 string[32]  compressorname; 
 template unsigned int(16)  depth = 0x0018; 
 int(16)  pre_defined = -1; 
} 
 
 // Audio Sequences 
 
class AudioSampleEntry(codingname) extends SampleEntry (codingname){ 
 const unsigned int(32)[2] reserved = 0; 
 template unsigned int(16) channelcount = 2; 
 template unsigned int(16) samplesize = 16; 
 unsigned int(16) pre_defined = 0; 
 const unsigned int(16) reserved = 0 ; 
 template unsigned int(32) samplerate = {timescale of media}<<16; 
}

aligned(8) class SampleDescriptionBox (unsigned int(32) handler_type) 
 extends FullBox(’stsd’, 0, 0){ 
 int i ; 
 unsigned int(32) entry_count; 
 for (i = 1 ; i u entry_count ; i++){ 
  switch (handler_type){ 
   case ‘soun’: // for audio tracks 
    AudioSampleEntry(); 
    break; 
   case ‘vide’: // for video tracks  
    VisualSampleEntry(); 
    break; 
   case ‘hint’: // Hint track 
    HintSampleEntry(); 
    break; 
  } 
 } 
}
```



In [153]:
class AudioSampleEntry(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Bytes(6),
            "data_reference_index" / Int16ub,
            "reserved1" / Bytes(8),
            "channelcount" / Int16ub,
            "samplesize" / Int16ub,
            "pre_defined" / Int16ub,
            "perserved2" / Bytes(2),
            "samplerate" / Aligned(4, Int16ub),
        )

        super().__init__('mp4a')


class HintSampleEntry(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Bytes(6),
            "data_reference_index" / Int16ub,
            "data" / Int8ub,
        )

        super().__init__()


class VisualSampleEntry(Box):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Bytes(6),
            "data_reference_index" / Int16ub,
            "pre_defined1" / Bytes(2),
            "reserved1" / Bytes(2),
            "pre_defined2" / Bytes(12),
            "width" / Int16ub,
            "height" / Int16ub,
            "horizresolution" / Aligned(4, Int16ub),
            "vertresolution" / Aligned(4, Int16ub),
            "reserved2" / Bytes(4),
            "frame_count" / Int16ub,
            "compressorname" / PascalString(Int8ub, 'utf8'),
            "depth" / Int16ub,
            "pre_defined3" / Bytes(2),
        )

        super().__init__('avc1')


class SampleDescriptionBox(FullBox):
    def __init__(self, handler_type):

        if handler_type == "vide":
            self.entry_fmt = VisualSampleEntry().fmt
        elif handler_type == "soun":
            self.entry_fmt = AudioSampleEntry().fmt
        elif handler_type == "hint":
            self.entry_fmt = HintSampleEntry().fmt

        self.body_fmt = Struct(
            "entry_count" / Int32ub, "entries" / Array(this.entry_count, self.entry_fmt)
        )

        super().__init__('stsd')


vide_stsd_offset = data.find(b"stsd") - 4
vide_stsd = SampleDescriptionBox("vide")
vide_stsd.init(data[vide_stsd_offset:])
print(vide_stsd)

soun_stsd_offset = data.find(b'stsd', vide_stsd_offset + 5) - 4
soun_stsd = SampleDescriptionBox('soun')
soun_stsd.init(data[soun_stsd_offset:])
print(soun_stsd)


box_name: stsd
Full Box Container: 
    size = 169
    type = u'stsd' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        entry_count = 1
        entries = ListContainer: 
            Container: 
                size = 153
                type = u'avc1' (total 4)
                large_size = None
                extended_type = None
                body = Container: 
                    reserved = b'\x00\x00\x00\x00\x00\x00' (total 6)
                    data_reference_index = 1
                    pre_defined1 = b'\x00\x00' (total 2)
                    reserved1 = b'\x00\x00' (total 2)
                    pre_defined2 = b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' (total 12)
                    width = 640
                    height = 360
                    horizresolution = 72
                    vertresolution = 72
                    reserved2 = b'\x00\x00\x00\x00' (total 4)
         

###### stco(Chunk Offset Box)

chunk在文件中的偏移量。针对小文件、大文件，有两种不同的box类型，分别是stco、co64，它们的结构是一样的，只是字段长度不同。

chunk_offset 指的是在文件本身中的 offset，而不是某个box内部的偏移。

在构建mp4文件的时候，需要特别注意 moov 所处的位置，它对于chunk_offset 的值是有影响的。有一些MP4文件的 moov 在文件末尾，为了优化首帧速度，需要将 moov 移到文件前面，此时，需要对 chunk_offset 进行改写。

定义如下:
```java
// Box Type: ‘stco’, ‘co64’
// Container: Sample Table Box (‘stbl’) Mandatory: Yes
// Quantity: Exactly one variant must be present

aligned(8) class ChunkOffsetBox
	extends FullBox(‘stco’, version = 0, 0) { 
	unsigned int(32) entry_count;
	for (i=1; i u entry_count; i++) {
		unsigned int(32)  chunk_offset;
	}
}

aligned(8) class ChunkLargeOffsetBox
	extends FullBox(‘co64’, version = 0, 0) { 
	unsigned int(32) entry_count;
	for (i=1; i u entry_count; i++) {
		unsigned int(64)  chunk_offset;
	}
}
```

In [154]:
class ChunkOffsetBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "chunk_offset_list" / Hex(Int32ub)[this.entry_count],
        )

        super().__init__('stco')

class ChunkLargeOffsetBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "chunk_offset_list" / Hex(Int64ub)[this.entry_count],
        )

        super().__init__('co64')


stco_offset = data.find(b'stco') - 4
stco = ChunkOffsetBox()
stco.init(data[stco_offset:])
print(stco)


box_name: stco
Full Box Container: 
    size = 13488
    type = u'stco' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        entry_count = 3368
        chunk_offset_list = ListContainer: 
            0x0001C056
            0x000232A2
            0x00023377
            0x00023555
            0x000236F5
            0x00023888
            0x0002395E
            0x00023AF2
            0x00023C85
            0x00023E1B
            0x00023EF0
            0x00024083
            0x000245D5
            0x000246AE
            0x00024843
            0x000249D6
            0x00024B69
            0x00024C43
            0x00024DD8
            0x00024F6B
            0x00025330
            0x00025410
            0x000255A8
            0x00025DE4
            0x00025EC6
            0x00026270
            0x0002746B
            0x00027AA2
            0x00027BEE
            0x00027DE0
            0x00027FBF
          

###### stsc(Sample To Chunk Box)

sample 以 chunk 为单位分成多个组。chunk的size可以是不同的，chunk里面的sample的size也可以是不同的。

- entry_count：有多少个表项（每个表项，包含first_chunk、samples_per_chunk、sample_description_index信息）；
- first_chunk：当前表项中，对应的第一个chunk的序号；
- samples_per_chunk：每个chunk包含的sample数；
- sample_description_index：指向 stsd 中 [sample description 的索引值](#stsdsample-description-box)

这里看个例子，这里表示的是：

- 序号1~15的chunk，每个chunk包含15个sample；
- 序号16的chunk，包含30个sample；
- 序号17以及之后的chunk，每个chunk包含28个sample；
- 以上所有chunk中的sample，对应的sample description的索引都是1；

| first_chunk | samples_per_chunk | sample_description_index |
| ----------- | ----------------- | ------------------------ |
| 1	 |15 | 1 |
| 16 |	30 | 1 |
| 17 |	28 | 1 |

定义如下:
```java
aligned(8) class SampleToChunkBox
	extends FullBox(‘stsc’, version = 0, 0) { 
	unsigned int(32) entry_count;
	for (i=1; i u entry_count; i++) {
		unsigned int(32) first_chunk;
		unsigned int(32) samples_per_chunk; 
		unsigned int(32) sample_description_index;
	}
}
```

In [155]:
class SampleToChunkBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "chunk_info_list"
            / Array(
                this.entry_count,
                Struct(
                    "first_chunk" / Int32ub,
                    "samples_per_chunk" / Int32ub,
                    "sample_description_index" / Int32ub,
                ),
            ),
        )

        super().__init__("stsc")

stsc_offset = data.find(b'stsc') - 4
stsc = SampleToChunkBox()
stsc.init(data[stsc_offset:])
print(stsc)


box_name: stsc
Full Box Container: 
    size = 40
    type = u'stsc' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        entry_count = 2
        chunk_info_list = ListContainer: 
            Container: 
                first_chunk = 1
                samples_per_chunk = 2
                sample_description_index = 1
            Container: 
                first_chunk = 2
                samples_per_chunk = 1
                sample_description_index = 1


###### stsz（Sample Size Boxes）

每个sample的大小（字节），根据 sample_size 字段，可以知道当前track包含了多少个sample（或帧）。

有两种不同的box类型，stsz、stz2。

stsz：

- sample_size：默认的sample大小（单位是byte），通常为0。如果sample_size不为0，那么，所有的sample都是同样的大小。如果sample_size为0，那么，sample的大小可能不一样。
- sample_count：当前track里面的sample数目。如果 sample_size==0，那么，sample_count 等于下面entry的条目；
- entry_size：单个sample的大小（如果sample_size==0的话）；

```java
aligned(8) class SampleSizeBox extends FullBox(‘stsz’, version = 0, 0) { 
	unsigned int(32) sample_size;
	unsigned int(32) sample_count;
	if (sample_size==0) {
		for (i=1; i u sample_count; i++) {
			unsigned int(32)  entry_size;
		}
	}
}
```

stz2：

- field_size：entry表中，每个entry_size占据的位数（bit），可选的值为4、8、16。4比较特殊，当field_size等于4时，一个字节上包含两个entry，高4位为entry[i]，低4位为entry[i+1]；
- sample_count：等于下面entry的条目；
- entry_size：sample的大小。

```java
aligned(8) class CompactSampleSizeBox extends FullBox(‘stz2’, version = 0, 0) { 
	unsigned int(24) reserved = 0;
	unisgned int(8) field_size;
	unsigned int(32) sample_count;
	for (i=1; i u sample_count; i++) {
		unsigned int(field_size) entry_size;
	}
}
```

In [156]:
class SampleSizeBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "sample_size" / Int32ub,
            "sample_count" / Int32ub,
            "sample_size_list" / Int32ub[this.sample_count],
        )

        super().__init__('stsz')


stsz_offset = data.find(b'stsz') - 4
stsz = SampleSizeBox()
stsz.init(data[stsz_offset:])
print(stsz)

class CompactSampleSizeBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "reserved" / Int24ub,
            "field_size" / Int8ub,
            "sample_count" / Int32ub,
            "sample_size_list" / BitsInteger(this.field_size)[this.sample_count],
        )

        super().__init__('stz2')


stz2_offset = data.find(b'stz2') - 4
if stz2_offset >= 0:
    stz2 = CompactSampleSizeBox()
    stz2.init(data[stz2_offset:])
    print(stz2)

box_name: stsz
Full Box Container: 
    size = 13496
    type = u'stsz' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        sample_size = 0
        sample_count = 3369
        sample_size_list = ListContainer: 
            28963
            107
            23
            98
            35
            23
            24
            23
            23
            25
            23
            23
            981
            27
            24
            23
            23
            27
            25
            23
            553
            44
            23
            1634
            40
            434
            4088
            1150
            142
            24
            24
            23
            23
            999
            52
            41
            40
            23
            25
            23
            23
            105
            25
            24
            23
        

###### stts（Decoding Time to Sample Box）

stts包含了 [DTS](#dtsdecoding-time-stamp-和-ptspresentation-time-stamp) 到sample number的映射表，主要用来推导每个帧的时长。

- entry_count：stts 中包含的entry条目数；
- sample_count：单个entry中，具有相同时长（duration 或 sample_delta）的连续sample的个数。
- sample_delta：sample的时长（以timescale为计量）

还是看例子，如下图，entry_count为3，
前250个sample的时长为1000，
第251个sample时长为999，
第252~283个sample的时长为1000。

>假设timescale为1000，则实际时长需要除以1000。

![图 1](../asset/ef97f55fd2fe04a68c5bafbc7912fac03c87f9ce143a914a572a9afb7fddd814.png)  


```java
aligned(8) class TimeToSampleBox extends FullBox(’stts’, version = 0, 0) {
	unsigned int(32)  entry_count;
	int i;
	for (i=0; i < entry_count; i++) {
		unsigned int(32)  sample_count;
		unsigned int(32)  sample_delta;
	}
}
```

In [157]:
class TimeToSampleBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "entry_list"
            / Array(
                this.entry_count,
                Struct(
                    "sample_count" / Int32ub,
                    "sample_delta" / Int32ub,
                ),
            ),
        )

        super().__init__("stts")


stts_offset = data.find(b"stts") - 4
stts = TimeToSampleBox()
stts.init(data[stts_offset:])
print(stts)


box_name: stts
Full Box Container: 
    size = 1488
    type = u'stts' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        entry_count = 184
        entry_list = ListContainer: 
            Container: 
                sample_count = 21
                sample_delta = 640
            Container: 
                sample_count = 1
                sample_delta = 624
            Container: 
                sample_count = 36
                sample_delta = 640
            Container: 
                sample_count = 1
                sample_delta = 624
            Container: 
                sample_count = 36
                sample_delta = 640
            Container: 
                sample_count = 1
                sample_delta = 624
            Container: 
                sample_count = 35
                sample_delta = 640
            Container: 
                sample_count = 1
                sample_delt

###### stss（Sync Sample Box）

mp4文件中，关键帧所在的sample序号。如果没有stss的话，所有的sample中都是关键帧。

- entry_count：entry的条目数，可以认为是关键帧的数目；
- sample_number：关键帧对应的sample的序号；（从1开始计算）

例子如下，第1、31、61、91、121...271个sample是关键帧。

![图 2](../asset/dde831b26f0efe11a5f445b2712133855e4f4c3aa6b953827ad6a642c36c2311.png)  

```java
aligned(8) class SyncSampleBox
   extends FullBox(‘stss’, version = 0, 0) {
   unsigned int(32)  entry_count;
   int i;
   for (i=0; i < entry_count; i++) {
      unsigned int(32)  sample_number;
   }
}
```


In [158]:
class SyncSampleBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "sample_number" / Int32ub[this.entry_count],
        )

        super().__init__('stss')


stss_offset = data.find(b'stss') - 4
stss = SyncSampleBox()
stss.init(data[stss_offset:])
print(stss)

box_name: stss
Full Box Container: 
    size = 124
    type = u'stss' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        entry_count = 27
        sample_number = ListContainer: 
            1
            126
            251
            376
            501
            626
            751
            876
            1001
            1126
            1251
            1376
            1501
            1626
            1751
            1876
            2001
            2126
            2251
            2376
            2501
            2626
            2751
            2876
            3001
            3126
            3251


###### ctts（Composition Time to Sample Box）

从解码（[DTS](#dtsdecoding-time-stamp-和-ptspresentation-time-stamp)）到渲染（[PTS](#dtsdecoding-time-stamp-和-ptspresentation-time-stamp)）之间的差值。

对于只有I帧、P帧的视频来说，解码顺序、渲染顺序是一致的，此时，ctts没必要存在。

对于存在B帧的视频来说，ctts就需要存在了。当PTS、DTS不相等时，就需要ctts了，公式为 CT(n) = DT(n) + CTTS(n) 。

```java
aligned(8) class CompositionOffsetBox extends FullBox(‘ctts’, version = 0, 0) { unsigned int(32) entry_count;
      int i;
   for (i=0; i < entry_count; i++) {
      unsigned int(32)  sample_count;
      unsigned int(32)  sample_offset;
   }
}
```

In [159]:
class CompositionOffsetBox(FullBox):
    def __init__(self):
        self.body_fmt = Struct(
            "entry_count" / Int32ub,
            "entry_list" / Array(this.entry_count, Struct(
                "sample_count" / Int32ub,
                "sample_offset" / Int32ub,
            ))
        )

        super().__init__('ctts')


ctts_offset = data.find(b'ctts') - 4
ctts = CompositionOffsetBox()
ctts.init(data[ctts_offset:])
print(ctts)

box_name: ctts
Full Box Container: 
    size = 25568
    type = u'ctts' (total 4)
    large_size = None
    extended_type = None
    version = 0
    flags = b'\x00\x00\x00' (total 3)
    body = Container: 
        entry_count = 3194
        entry_list = ListContainer: 
            Container: 
                sample_count = 1
                sample_offset = 1920
            Container: 
                sample_count = 1
                sample_offset = 2560
            Container: 
                sample_count = 1
                sample_offset = 1280
            Container: 
                sample_count = 1
                sample_offset = 1920
            Container: 
                sample_count = 1
                sample_offset = 6400
            Container: 
                sample_count = 1
                sample_offset = 3200
            Container: 
                sample_count = 1
                sample_offset = 1280
            Container: 
                sample_count = 1
               

# 如何计算某一帧的 Sample 偏移位置

步骤如下: 
1. 将 PTS 转换到媒体对应的时间坐标系。
2. 根据 stts 计算某个 PTS 对应的 Sample 序号。
3. 根据 stsc 计算 Sample 序号存放在哪个 Chunk 中。
4. 根据 stco 获取对应 Chunk 在文件中的偏移位置。
5. 根据 stsz 获取 Sample 在 Chunk 内的偏移位置并加上第 4 步获取的偏移，计算出 Sample 在文件中的偏移。

例如下面代码计算 3.56s 的 Sample 偏移位置

In [167]:
pts = 3.56
print(f"pts: {pts}")
# 1. 通过 mdhd 中的 timescale 将 pts 转换成文件中的时间坐标系
pts_in_media = pts * mdhd.body.timescale
print(f"pts_in_media: {pts_in_media}")

# 2. 根据 stts 计算某个 PTS 对应的 Sample 序号。
import math


def getSampleIdxByPts(pts_in_media):
    cur_delta = 0
    cur_sample = 0
    for sample_delta in stts.body.entry_list:
        cur_sample += sample_delta.sample_count
        cur_delta += sample_delta.sample_count * sample_delta.sample_delta
        # print(sample_delta)
        if cur_delta >= pts_in_media:
            ret = math.floor(
                cur_sample - (cur_delta - pts_in_media) / sample_delta.sample_delta
            )
            return ret


sample_idx = getSampleIdxByPts(pts_in_media)
print(f"sample_idx: {sample_idx}")

# 3. 根据 stsc 计算 Sample 序号存放在哪个 Chunk 中。
def getChunkAndOffset(sample_idx):
    total_sample = 0
    for i in range(len(stsc.body.chunk_info_list)):
        cur_chunk_block = stsc.body.chunk_info_list[i]

        if i != len(stsc.body.chunk_info_list) - 1:
            next_chunk_block = stsc.body.chunk_info_list[i + 1]

            cur_chunk_block_sample_count = (
                next_chunk_block.first_chunk - cur_chunk_block.first_chunk
            ) * cur_chunk_block.samples_per_chunk

            if sample_idx > total_sample + cur_chunk_block_sample_count:
                total_sample += cur_chunk_block_sample_count
                continue

        sample_idx_in_chunk_block = sample_idx - total_sample
        chunk_idx = cur_chunk_block.first_chunk + (
            math.ceil(sample_idx_in_chunk_block / cur_chunk_block.samples_per_chunk) - 1
        )
        sample_idx_in_chunk = sample_idx_in_chunk_block - (chunk_idx - cur_chunk_block.first_chunk) * cur_chunk_block.samples_per_chunk

        return chunk_idx, sample_idx_in_chunk, sample_idx - sample_idx_in_chunk + 1


chunk_idx, sample_idx_in_chunk, chunk_first_sample_idx = getChunkAndOffset(sample_idx)

print(f"chunk_idx: {chunk_idx}")
print(f"sample_idx_in_chunk: {sample_idx_in_chunk}")
print(f"chunk_first_sample_idx: {chunk_first_sample_idx}")

# 4. 根据 stco 获取对应 Chunk 在文件中的偏移位置。
chunk_offset = stco.body.chunk_offset_list[chunk_idx - 1]
print(f"chunk_offset: {chunk_offset}")

# 5. 根据 stsz 获取 Sample 在 Chunk 内的偏移位置并加上第 4 步获取的偏移，计算出 Sample 在文件中的偏移。
sample_size = stsz.body.sample_size_list[sample_idx - 1]
sample_offset_in_chunk = 0
for i in range(chunk_first_sample_idx - 1, sample_idx - 1):
    sample_offset_in_chunk += stsz.body.sample_size_list[i]
print(f"sample_offset_in_chunk: {hex(sample_offset_in_chunk)}")
print(f"sample_size: {sample_size}")

sample_offset_in_file = chunk_offset + sample_offset_in_chunk
print(f"sample_offset_in_file: {hex(sample_offset_in_file)}")

# for sample_info in ctts.body.entry_list:


# print(ctts)


pts: 3.56
pts_in_media: 56960.0
sample_idx: 89
chunk_idx: 88
sample_idx_in_chunk: 1
chunk_first_sample_idx: 89
chunk_offset: 0x0002EADE
sample_offset_in_chunk: 0x0
sample_size: 23
sample_offset_in_file: 0x2eade
